# Setup

In [ ]:
import numpy as np
import pandas as pd
import re

## Wikipedia

In [ ]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=7e7aefccce328c8b1eabb5e26b6a61f8960012469fdb08dd5f5523a28bd79f63
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [ ]:
import wikipedia

## Project Gutenberg

In [ ]:
!pip install gutenbergpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for httpsproxy-urllib2: filename=httpsproxy_urllib2-1.0-py3-none-any.whl size=29269 sha256=96e258a798b71f6da59b6e52293ac6398b7793297acbb03b1de1d350341ece97
  Stored in directory: /root/.cache/pip/wheels/91/04/37/b7080e0cfe33a8bc79ff0082159cea767f9145a5a713b501a3
Successfully built httpsproxy-urllib2


In [ ]:
import gutenbergpy.textget

In [ ]:
# This part of code takes 15-20 mins, basically getting geoguessr cache to allow us to query it
# TODO save cache locally so that it need only be queried once
from gutenbergpy.gutenbergcache import GutenbergCache
GutenbergCache.create(refresh=True, download=True, unpack=True, parse=True, cache=True, deleteTemp=True)

Deleting old files
 Extracting  rdf-files.tar.bz2 : [###################]took 438.402402
 Processing progress: 68440 / 68441 : [###################]RDF PARSING took 406.8654992580414
 SQLite progress : [###################]sql took 135.194983
Deleting temporary files
Done


In [ ]:
cache  = GutenbergCache.get_cache()

## Web Scraping

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.9 MB/s 
     |████████████████████████████████| 358 kB 20.6 MB/s 
     |████████████████████████████████| 139 kB 17.4 MB/s 
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 4.1 MB 31.6 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--headless") 
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument(
    "--blink-settings=imagesEnabled=false"
)  # Do not load images

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

# Functions

In [ ]:
def get_book_triple(title):
  try:
    full_text = get_book_text(title)
  except:
    full_text="unknown error"

  try:
    sum = get_book_sum(title)
    sum = ""
  except:
    sum = "unknown error"

  try:
    characters = get_book_chars(title)
  except:
    characters = []

  return {"title":title,
          "full_text":full_text,
          "summary":sum,
          "characters":characters}

def cleantext(text):
  text = text.replace("\n"," ")
  text = text.split(" ")
  text = list(filter(lambda a: a != "", text))
  return " ".join(text)

# Query gutenberg, get first result and extract full text (cleaned up a little)
def get_book_text(title):
  q_results = cache.query(downloadtype=['application/plain','text/plain','text/html; charset=utf-8'],titles=[title],languages=["en"])
  if not q_results:
    return "no results found"

  for book_ID in q_results:
    try:
      raw_book = gutenbergpy.textget.get_text_by_id(book_ID)
      clean_book  = gutenbergpy.textget.strip_headers(raw_book)
      clean_book = clean_book.decode("utf-8")
      if(len(clean_book)>200):
        clean_book = cleantext(clean_book)
        return clean_book
    except:
      pass
  
  return "No valid book found"

# Opens exact Page for wikipedia, finds and extract summary section
# TODO allow wikipedia querying to allow for more flexible page titles
def get_book_sum(title):
  summary = ""
  bookPage = wikipedia.page(title, auto_suggest=False)

  summary = bookPage.section("Plot")
  if(summary == None): summary = bookPage.section("Summary")
  if(summary == None): summary = bookPage.section("Plot summary")
  if(summary == None): summary = bookPage.section("Synopsis")
  if(summary == None): summary = 'Summary Section Not found'
  if(summary == ""): summary = "Sumary Section empty"
  
  return cleantext(summary)

# Create character lists from The book page
def get_characters(webbed_driver):
        characters = []
        listelems = webbed_driver.find_elements(
            by = By.XPATH,
            value ='//a[contains(@href, "/characters/")]' # character names are always links to another webpage, with /characters/ so we use this property to find them in the page
        )
        if listelems:
          for e in listelems:
            characters.append(e.get_attribute("innerText"))
          return characters
        else:
          return []

# find the book's goodreads page as first result from text query 
def get_book_chars(book_name):
  book_name = book_name.replace(" ", "+")
  driver.get("https://www.goodreads.com/search?q="+book_name)
  elems = driver.find_elements(
            by = By.CLASS_NAME,
            value ='bookTitle'
        )
  if elems:
    book_url=elems[0].get_attribute("href")
    driver.get(book_url)
    return get_characters(driver)
  else:
    return []

In [ ]:
# Goodreads Character List:
get_book_chars("Pride and Prejudice")

In [ ]:
cache.query(titles=["Pride and Prejudice"])

In [ ]:
# Test
get_book_triple("Pride and Prejudice")

# Book list

In [ ]:
guntenberg_list = "Pride and Prejudice by Jane Austen (37715)~Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley (22856)~Alice's Adventures in Wonderland by Lewis Carroll (22619)~The Adventures of Sherlock Holmes by Arthur Conan Doyle (20667)~Moby Dick; Or, The Whale by Herman Melville (14879)~Dracula by Bram Stoker (14841)~Noli Me Tangere by José Rizal (14263)~Ulysses by James Joyce (13198)~The Picture of Dorian Gray by Oscar Wilde (13033)~A Tale of Two Cities by Charles Dickens (12521)~The Great Gatsby by F. Scott Fitzgerald (12393)~The Yellow Wallpaper by Charlotte Perkins Gilman (12035)~The Strange Case of Dr. Jekyll and Mr. Hyde by Robert Louis Stevenson (10334)~Great Expectations by Charles Dickens (9930)~War and Peace by graf Leo Tolstoy (9080)~A Doll's House : a play by Henrik Ibsen (8995)~The Importance of Being Earnest: A Trivial Comedy for Serious People by Oscar Wilde (8659)~Jane Eyre: An Autobiography by Charlotte Brontë (8648)~Japanese Girls and Women by Alice Mabel Bacon (8518)~Crime and Punishment by Fyodor Dostoyevsky (8459)~Metamorphosis by Franz Kafka (8253)~The Count of Monte Cristo, Illustrated by Alexandre Dumas (8054)~A Modest Proposal by Jonathan Swift (7918)~Grimms' Fairy Tales by Jacob Grimm and Wilhelm Grimm (7791)~Adventures of Huckleberry Finn by Mark Twain (7457)~The Adventures of Tom Sawyer, Complete by Mark Twain (7329)~A Christmas Carol in Prose; Being a Ghost Story of Christmas by Charles Dickens (7248)~The Iliad by Homer (7200)~Heart of Darkness by Joseph Conrad (7108)~Simple Sabotage Field Manual by United States. Office of Strategic Services (6988)~The Prince by Niccolò Machiavelli (6911)~The Romance of Lust: A classic Victorian erotic novel by Anonymous (6666)~The Philippines a Century Hence by José Rizal (6658)~Tractatus Logico-Philosophicus by Ludwig Wittgenstein (6544)~Treasure Island by Robert Louis Stevenson (6472)~Little Women by Louisa May Alcott (6436)~Anne of Green Gables by L. M. Montgomery (6436)~The Wonderful Wizard of Oz by L. Frank Baum (6358)~The Scarlet Letter by Nathaniel Hawthorne (6210)~Don Quixote by Miguel de Cervantes Saavedra (6120)~The Prophet by Kahlil Gibran (5863)~Dubliners by James Joyce (5844)~The Odyssey by Homer (5759)~Les Misérables by Victor Hugo (5711)~Winnie-the-Pooh by A. A. Milne (5685)~The Hound of the Baskervilles by Arthur Conan Doyle (5641)~Walden, and On The Duty Of Civil Disobedience by Henry David Thoreau (5519)~The Brothers Karamazov by Fyodor Dostoyevsky (5472)~Emma by Jane Austen (5355)~Peter Pan by J. M. Barrie (5335)~Autobiography of Benjamin Franklin by Benjamin Franklin (5316)~The Kama Sutra of Vatsyayana by Vatsyayana (5285)~The Time Machine by H. G. Wells (5269)~Beyond Good and Evil by Friedrich Wilhelm Nietzsche (5209)~The Jungle Book by Rudyard Kipling (5093)~The Republic by Plato (5037)~Wuthering Heights by Emily Brontë (5011)~Anna Karenina by graf Leo Tolstoy (4979)~Oliver Twist by Charles Dickens (4979)~The Souls of Black Folk by W. E. B. Du Bois (4932)~The War of the Worlds by H. G. Wells (4660)~The Reign of Greed by José Rizal (4424)~Persuasion by Jane Austen (4255)~Sense and Sensibility by Jane Austen (4201)~A Study in Scarlet by Arthur Conan Doyle (4195)~Thus Spake Zarathustra: A Book for All and None by Friedrich Wilhelm Nietzsche (4186)~Beowulf: An Anglo-Saxon Epic Poem (4168)~David Copperfield by Charles Dickens (4058)~The Works of Edgar Allan Poe — Volume 2 by Edgar Allan Poe (4018)~Essays of Michel de Montaigne — Complete by Michel de Montaigne (3896)~Moby Multiple Language Lists of Common Words by Grady Ward (3819)~Old Granny Fox by Thornton W. Burgess (3814)~The King in Yellow by Robert W. Chambers (3790)~Meditations by Emperor of Rome Marcus Aurelius (3788)~The King James Version of the Bible (3744)~Gulliver's Travels into Several Remote Nations of the World by Jonathan Swift (3737)~The Call of the Wild by Jack London (3674)~The Legend of Sleepy Hollow by Washington Irving (3671)~The Awakening, and Selected Short Stories by Kate Chopin (3662)~Pygmalion by Bernard Shaw (3601)~The Mysterious Affair at Styles by Agatha Christie (3561)~Siddhartha by Hermann Hesse (3509)~Uncle Tom's Cabin by Harriet Beecher Stowe (3481)~A Pickle for the Knowing Ones by Timothy Dexter (3415)~Second Treatise of Government by John Locke (3411)~The Secret Garden by Frances Hodgson Burnett (3396)~An Index of The Divine Comedy by Dante by Dante Alighieri (3258)~The Social Cancer: A Complete English Version of Noli Me Tangere by José Rizal (3207)~Leviathan by Thomas Hobbes (3173)~Calculus Made Easy by Silvanus P. Thompson (3167)~Carmilla by Joseph Sheridan Le Fanu (3137)~Anthem by Ayn Rand (3105)~Josefine Mutzenbacher by Felix Salten (3050)~The Happy Prince, and Other Tales by Oscar Wilde (3047)~Narrative of the Life of Frederick Douglass, an American Slave by Frederick Douglass (3041)~Through the Looking-Glass by Lewis Carroll (3040)~Notes from the Underground by Fyodor Dostoyevsky (3025)~Around the World in Eighty Days by Jules Verne (3014)~Memoirs of Extraordinary Popular Delusions and the Madness of Crowds by Charles Mackay (2942)"

thegreatestbooks_list = "In Search of Lost Time~Ulysses~Don Quixote~The Great Gatsby~Moby Dick~War and Peace~Hamlet~The Odyssey~Madame Bovary~The Divine Comedy~The Brothers Karamazov~Crime and Punishment~Wuthering Heights~Pride and Prejudice~The Adventures of Huckleberry Finn~Anna Karenina~Alice's Adventures in Wonderland~The Iliad~Heart of Darkness~Great Expectations~One Thousand and One Nights~The Trial~The Red and the Black~Middlemarch~Gulliver's Travels~Mrs. Dalloway~Jane Eyre~The Aeneid~The Sun Also Rises~David Copperfield~Tristram Shandy~Leaves of Grass~The Magic Mountain~A Portrait of the Artist as a Young Man~Oedipus the King~Candide~The Idiot~Les MisÃ©rables~A Passage to India~Emma~The Metamorphosis~The Portrait of a Lady~Frankenstein~Antigone~The Possessed~The Age of Innocence~Dead Souls~The Good Soldier~Orlando: A Biography~The Canterbury Tales~Vanity Fair~The Castle~A Sentimental Education~The Scarlet Letter~The Charterhouse of Parma~Paradise Lost~Gargantua and Pantagruel~Faust~The Flowers of Evil~Decameron~Tess of the d'Urbervilles~Robinson Crusoe~Tom Jones~King Lear~Father Goriot~Sons and Lovers~Fathers and Sons~Oresteia~Bleak House~A Season in Hell~Howards End~Little Women~Fairy Tales and Stories~Lord Jim~Cousin Bette~The Tale of Genji~Treasure Island~The Count of Monte Cristo~Epic of Gilgamesh~Nostromo~The Tempest~Dangerous Liaison~Buddenbrooks~My Antonia~The Call of the Wild~First Folio~The Wind in the Willows~Metamorphoses~Oedipus at Colonus~Medea~Macbeth~Lost Illusions~The Ambassadors~Confessions of Zeno~Dracula~Women in Love~The Complete Sherlock Holmes~The Hound of the Baskervilles~Eugenie Grandet~A Room With a View~Twenty Thousand Leagues Under the Sea~The Princess of Cleves~The Strange Case of Dr. Jekyll and Mr. Hyde~The Pickwick Papers~Jude the Obscure~The House of Mirth~The Republic~Clarissa~The Good Soldier Svejk~Electra~The Picture of Dorian Gray~La Regenta~Winnie the Pooh~Kim~A Doll's House~Hunger~The Three Musketeers~De Rerum Natura~An American Tragedy~A Tale of Two Cities~The Jungle~The Time Machine~Parade's End~Dubliners~Through the Looking Glass~Thus Spake Zarathustra~The Mill on the Floss~Ethan Frome~Prometheus Bound~Persuasion~Siddhartha~The Suppliants~The Persians~Seven Against Thebes~Of Human Bondage~Moll Flanders~Philoctetes~Ajax~Women of Trachis~The Red Badge of Courage~Tarzan of the Apes~The Counterfeiters~Jacques the Fatalist and His Master~The Confusions of Young TÃ¶rless~Main Street~War of the Worlds~Our Mutual Friend~Stories of Guy de Maupassant~The Man of Property~In Chancery~To Let~Awakening~Indian Summer of a Forsyte~Lysistrata~The Complete Poetry and Prose of William Blake~Bouvard et PÃ©cuchet~The Woman in White~Bartleby the Scrivener~Maldoror (Les Chants de Maldoror)~The Rainbow~Kristin Lavransdatter~The Duino Elegies~The Wonderful Wizard of Oz~Household Tales~Death in Venice~Oblomov~The Death of Ivan Ilyich~The Mayor of Casterbridge~Journey to the West~The Adventures of Oliver Twist~The Birds~Three Sisters~The Hunchback of Notre-Dame~The Posthumous Memoirs of Bras Cubas~Way of All Flesh~The Old Wives' Tale~Wings of the Dove~The Clouds~The Tale of Peter Rabbit~Notes from the Underground~Ivanhoe~Dream of the Red Chamber~Pilgrim's Progress~Le Morte d'Arthur~The Last of the Mohicans~ThÃ©rÃ¨se Raquin~Germinal~Uncle Tom's Cabin~Winesburg, Ohio~The Sorrows of Young Werther~The Last Chronicle of Barset~The Bacchae~The Betrothed~A Hero of Our Time~We~The Custom of the Country~Carry On, Jeeves~The Cherry Orchard~New Grub Street: A Novel~The Poems of John Keats~PhÃ¨dre~Rubaiyat of Omar Khayyam~Silas Marner~The Way We Live Now~Six Characters in Search of an Author~The Secret Agent~The Nibelungenlied~In Our Time~The Lusiad~Sister Carrie~Barchester Towers~The Rime of the Ancient Mariner~The Awakening~Dom Casmurro~Songs of Innocence and Experience~Arrowsmith~Othello~Riddle of the Sands~Babbitt~A Midsummer Night's Dream~The Notebooks of Malte Laurids Brigge~The Adventures of Tom Sawyer~Cyrano de Bergerac~Trojan Women~The Sonnets~The Prelude~Pygmalion~The Moonstone~The Odes~Hippolytus~Don Juan: A Poem~The Turn of the Screw~The Diary of a Nobody~The Thirty-Nine Steps~Poems by Machado~The House of the Seven Gables~Celestina~Eugene Onegin~Hedda Gabler~The Murder of Roger Ackroyd~Aesop's Fables~Daniel Deronda~Tartuffe~Fortunata and Jacinta~Jakob Von Gunten~The Golden Bowl~Lyrical Ballads~Adolphe~Utopia~Tristan~Life Is a Dream~Anne of Green Gables~The Poems of Francois Villon~Collected Poems~The Complete Poetry and Selected Prose of John Donne~Peter Pan~Beowulf~Kokoro~Platero~Rashomon and Seventeen Other Stories~Romance of the Three Kingdoms~The Prophet~Lazarillo de Tormes~Sense and Sensibility~Alcools~A Christmas Carol~The Private Memoirs and Confessions of a Justified Sinner~The Red Room~HemsÃ¶borna~The Alberta Trilogy~Zuleika Dobson~The Country of the Pointed Firs~Romeo and Juliet~Uncle Vanya~Little Dorrit~Manon Lescaut~Ben-Hur~Hyperion~Simplicius Simplicissimus~The Life and Opinions of the Tomcat Murr~Elective Affinities~Green Henry~The Tenant of Wildfell Hall~Growth of the Soil~The Stechlin~Aus dem Leben eines Taugenichts~Diary of a Madman and Other Stories~Robert Browning's Poetry~Fragments~The Magnificent Ambersons~The Maias: Episodes from Romantic Life~The Golden Ass (Metamorphoses): Or Metamorphoses~BÃ©rÃ©nice~Billy Budd~A Lost Lady~Cranford~The Lady with the Dog~Richard III~The Kreutzer Sonata: And Other Stories~Home and the World~The Castle of Otranto: A Gothic Story~Blindness~Against Nature~The Importance of Being Earnest~Three Lives~Poems of Rumi~Professor Unrat~The Recognition of Sakuntala~Wilhelm Meister's Apprenticeship~Michael Kohlhaas~Effi Briest~The Making of Americans~Under Satan's Sun~The Ragged Trousered Philanthropists~The Water Margin: Outlaws of the Marsh~Oroonoko~Hadrian the Seventh~Gentlemen Prefer Blondes: The Illuminating Diary of a Professional Lady~The Narrow Road to the Deep North~The Warden~Riders of the Purple Sage~The Complete Poetry and Essential Prose of John Milton~The Misanthrope~Twice-Told Tales~The Poems of Walt Whitman~The Collected Works of St. John of the Cross~Le Cid~Fables of La Fontaine~Petrarch's Songbook~The Faerie Queene~The Complete Poems of Samuel Taylor Coleridge~Saint Joan~The Master Builder~The Wild Duck~El BuscÃ³n~Phaedrus~The Interior Castle~Selected Poems~Poems of Friedrich HÃ¶lderlin~Twelfth Night: Or, What You Will~Les Filles du feu~Soledades~The Book of the City of Ladies~Bostan~Njal's Saga~Masnavi~Daphnis and Chloe~Children of Gebelawi~Parzival~The Complete Writings of Alfred de Musset~The Figaro Trilogy: The Barber of Seville, The Marriage of Figaro, The Guilty Mother~She Stoops to Conquer~Stories of Washington Irving~Germany, a Winter Tale~Les Tragiques~Les DestinÃ©es~Calila e Dimna~Coplas por la muerte de su padre~Ubu Roi~Daisy Miller~The Return of the Native~Carmen~The Poems of Percy Bysshe Shelley~The Wanderer~Marivaux: Three Plays~La lÃ©gende des siÃ¨cles~Gospels~GuzmÃ¡n de Alfarache~Manhattan Transfer~Calligrammes~Trilce~Rimas~Lad: a Dog~Demian~Sonnets~Poems of GÃ³ngora~Illuminations~Bohemian Lights~The Shadow Line~Wry-Blue Loves: Les Amours Jaunes~Les Contemplations~The Jungle Book~Villette~News from Nowhere~The Man of Feeling~The Enormous Room~The Enchanted Wanderer~The Fruits of the Earth~Sandokan~What Maisie Knew~The Nun~Forest of the Hanged~The Crime of Father Amaro~Eclipse of the Crescent Moon~North and South~I Malavoglia~Compassion~Uncle Silas~Tarr~Under the Yoke~The History of Rasselas, Prince of Abissinia~GÃ¶sta Berling's Saga~The Water-Babies, A Fairy Tale for a Land Baby~Bel Ami~Journey to the Center of the Earth~The Vicar of Wakefield~Solitude~The Green Hat~Indian Summer~By the Open Sea~The Artamonov Business~The Immoralist~None but the Brave~The Viceroys~The Charwoman's Daughter~Fanny Hill~La Celestina~Thomas of Reading, Or, The Sixe Worthie Yeomen of the West~Joseph Andrews~Julie, or the New Heloise~Justine~The Adventures of Caleb Williams~Mansfield Park~Camilla~In a Glass Darkly~Phineas Finn~Mother~Around the World in Eighty Days~Amok~Pepita Jimenez~The Life of Lazarillo de Tormes~Quo Vadis~Some Experiences of an Irish R. M.~The House on the Borderland~Anton Reiser~Erewhon~The Underdogs~Max Havelaar~ThaÃ¯s~Under Fire~The Unfortunate Traveller~The House of Ulloa~Le Diable au corps~Pharaoh~Locus Solus~Camera Obscura~Memoirs of Martinus Scriblerus~Pallieter~Une vie~Impressions of Africa~The Pit and the Pendulum~Torrents of Spring~The Nose~King Solomon's Mines~The Adventures of Peregrine Pickle~Far from the Madding Crowd~Melmoth the Wanderer~The Professor's House~The Tale of the Bamboo Cutter~Strait is the Gate~Il Piacere~Rob Roy~The Fall of the House of Usher~FantÃ´mas~The Monk~Castle Rackrent~La BÃªte humaine~Pierre et Jean~Vathek~The Island of Doctor Moreau~Nana~SenilitÃ~Heinrich of Ofterdingen~Down There~Eline Vere~Adam Bede~Tirant Lo Blanc~Life and Death of Harriett Frean~MartÃ­n Fierro~Evelina~The 120 Days of Sodom~The Devil's Pool~Crome Yellow~A Sentimental Journey Through France and Italy~The Travels of Persiles and Sigismunda~Life of Christ~King Lear of the Steppes~Amadis of Gaul~The Return of the Soldier~The Quest~Marius the Epicurean~Drunkard~Claudine~The Lion of Flanders~One, No One and One Hundred Thousand~The Mysteries of Udolpho~The Expedition of Humphry Clinker~Love in Excess; or, The Fatal Enquiry~Hell~The Female Quixote~Black Beauty: The Autobiography of a Horse~White Fang~Dombey and Son~The Beautiful and Damned~Tender Buttons~The Eustace Diamonds~My Book House~Mysteries~The Innocence of Father Brown~Amelia~The School for Wives~The Would-Be Invalid~Renascence and Other Poems~War Poems of Siegfried Sassoon~Shahnameh~The Would-Be Gentleman~The Prince and the Pauper: A Tale for Young People of All Ages~Don Juan~Alcestis~The Prussian Officer~The Beast in the Jungle~The Inimitable Jeeves~The Blithedale Romance~The Miser~The Wild Swans at Coole~The Plum in the Golden Vase~The Mysterious Affair at Styles~The Merry Adventures of Robin Hood~The Emperor of Portugallia~The Serious Game~Doctor Glas~The Queen's Tiara~Prufrock and Other Observations~The Voyages of Doctor Dolittle~The Song of Hiawatha~The Scarlet Plague~The Warlord of Mars~The Cloister and the Hearth~The Secret Garden~Emily of New Moon~First Love~Poems of Alfred Lord Tennyson~SiebenkÃ¤s~The Kalevala~Stories of Adalbert Stifter~SchatzkÃ¤stlein des Rheinischen Hausfreundes~Psmith, Journalist~Stories from the Lord Keuner~Stories~The Tootinameh; Or, Tales of a Parrot~Henry V~Lenz~Much Ado about Nothing~Mr Standfast~Abu Telfan, Return from the Mountains of the Moon~Le Roman de la Rose~The Loyal Subject~Nightmare Abbey~Under Western Eyes~The Phantom of the Opera~The Talisman~Candida: a Pleasant Play~The Sea-Wolf~Tales from the Kathasaritsagara~Three Men in a Boat~Far Tortuga~The Iron Heel~The Railway Children~The Chronicles of Barsetshire~The Murders in the Rue Morgue~Petersburg~Moravagine~The Narrative of Arthur Gordon Pym~The Manuscript Found in Saragossa~Flatland: A Romance of Many Dimensions~As You Like it~Kidnapped~Venus in Furs~Julius Caesar~The Poems of Sappho~The Poems of Wilfred Owen~Selected Poems of Lord Byron~O Pioneers!~The Bostonians~This Side of Paradise~Where Angels Fear to Tread~Greenmantle~The Man Who Was Thursday~The Theogony~A Little Princess~So Big~The Able McLaughlins~Early Autumn~His Family~The Complete Fairy Tales of Charles Perrault~Resurrection: A Novel~Alice Adams~One of Ours~The Gambler~The American~Cane~The Trickster of Seville and His Guest of Stone~Waverley~The Illustrious House of Ramires~The Spoils of Poynton~Miss Julie~John Wesley's Sermons: An Anthology~The Invisible Man~Nicholas Nickleby~Everyman~The Pioneers~Volpone~The Winter's Tale~New Atlantis~The Rape of the Lock~The Twelve Labours of Hercules~The Quest for the Golden Fleece~A Reckless Character and Other Stories~The Song of Roland~An Enemy of the People~Spoon River Anthology~Works and Days~Reynard the Fox~Sonnets from the Portuguese~The Master of Ballantrae: A Winter's Tale~Poem of the Cid~The Poems of Emily Bronte~An Essay on Man~The Father~The Orlando Furioso~The Protocols of the Elders of Zion~The Works of Walter Savage Landor~Poems of Matsuo BashÅ~Penguin Island~Orpheus and Eurydice~Benito Cereno~Stories of Nikolai Gogol~The Poetry of Alexander Pushkin~Thyestes~She~Splendors and Miseries of Courtesans~The Last Days of Mankind~Nathan the Wise~Don Carlos: Infante of Spain, a Drama in Five Acts~Edward Lear's Book of Nonsense~Tendrils of Vines~Capital of Pain~Martin Eden~Ballad of the Salt Sea~Life in the Far West~The Wonderful Adventures of Nils~Lady into Fox~Autobiography of an Ex-Colored Man~The Voyage Out~Maggie: A Girl of the Streets~Hagoromo: A Celestial Robe~Christina Rossetti: The Complete Poems~The Sign of Four~Apu Ollontay~Struwwelpeter, Or, Merry Rhymes and Funny Pictures...~Cheri~The Three Hundred Tang Poems~Hard Times~An Occurrence at Owl Creek Bridge~The History of Mr. Polly~The Merchant of Venice~Le Bateau Ivre~The Works of George Peele~The Taming of the Shrew~Collected Short Stories of Saki~The Continental Op~Foliage: Or, Poems Original and Translated~Henry IV, Part I~Lolly Willowes~The Complete Short Stories of Jack London~Sybil: Or The Two Nations~Selected Poems~The Dyskolos~The Tragical History of Doctor Faustus~Kabbalah~If: A Father's Advice to His Son~The Hunting of the Snark: An Agony, in Eight Fits~MeghadÅ«ta~The Complete Plays of Jean Racine~The Human Comedy~The Works of Moliere~Selected Plays of Henrick Ibsen~The Forsyte Saga~Plays"

#titles= guntenberg_list.split("~")
titles = thegreatestbooks_list.split("~")

In [ ]:
len(titles)

689

In [ ]:
#for i in range(len(titles)):
#  titles[i] = titles[i].split(" by")[0]

# Create Dataset

In [ ]:
book_triples = []

from IPython.display import clear_output 

numbooks = str(len(titles))
curbook =0

for title in titles:
  clear_output()
  curbook+=1
  print("processing book: " + str(curbook) + " / " + numbooks)
  book_triples.append(get_book_triple(title))
  clear_output()

df = pd.DataFrame(book_triples)

In [ ]:
df.head(50)

,title,full_text,summary,characters
0,In Search of Lost Time,no results found,,[]
1,Ulysses,[Illustration] Ulysses by James Joyce Contents...,,"[Stephen Dedalus, Leopold Bloom, Marion ""Molly..."
2,Don Quixote,bookcover.jpg (230K) Full Size spine.jpg (152K...,,"[Sancho Panza, Don Quijote de la Mancha]"
3,The Great Gatsby,\t\t\t The Great Gatsby \t\t\t\t by \t\t\t F. ...,,"[Nick Carraway, Jay Gatsby, Daisy Buchanan, To..."
4,Moby Dick,no results found,,"[Elijah, Stubb, Captain Ahab, Ishmael, Queeque..."
5,War and Peace,WAR AND PEACE By Leo Tolstoy/Tolstoi Contents ...,,"[Pierre Bezukhov, Natasha Rostova, Nikolai Ros..."
6,Hamlet,Project Gutenberg's Etext of Shakespeare's The...,,"[Horatio, Polonius, Laertes, Gertrude, Marcell..."
7,The Odyssey,cover The Odyssey by Homer Translated by Alexa...,,"[Odysseus, Penelope (wife of Odysseus), Helen ..."
8,Madame Bovary,Madame Bovary By Gustave Flaubert Translated f...,,"[Emma Bovary, Charles Bovary, Monsieur Homais,..."
9,The Divine Comedy,no results found,,"[Virgilio (Publius Vergilius Maro), Lucifer, O..."


In [ ]:
mask = (df['full_text'].str.len() > 200) & (df['summary'].str.len() > 30)
df_good = df.loc[mask]

In [ ]:
df_chars = df.loc[df['full_text'].str.len() > 200]
df_chars["none"] = df_chars["characters"].map(lambda x: len(x)>1)
df_chars = df_chars.loc[df_chars["none"] == True]
df_chars = df_chars.drop(columns = "none")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_chars

,title,full_text,summary,characters
1,Ulysses,[Illustration] Ulysses by James Joyce Contents...,,"[Stephen Dedalus, Leopold Bloom, Marion ""Molly..."
2,Don Quixote,bookcover.jpg (230K) Full Size spine.jpg (152K...,,"[Sancho Panza, Don Quijote de la Mancha]"
3,The Great Gatsby,\t\t\t The Great Gatsby \t\t\t\t by \t\t\t F. ...,,"[Nick Carraway, Jay Gatsby, Daisy Buchanan, To..."
5,War and Peace,WAR AND PEACE By Leo Tolstoy/Tolstoi Contents ...,,"[Pierre Bezukhov, Natasha Rostova, Nikolai Ros..."
6,Hamlet,Project Gutenberg's Etext of Shakespeare's The...,,"[Horatio, Polonius, Laertes, Gertrude, Marcell..."
...,...,...,...,...
610,The Pioneers,"THE PIONEERS, BY R.M. BALLANTYNE. PREFACE. Sir...",,"[Meriwether Lewis, William Clark, Thomas Jeffe..."
618,The Song of Roland,The Song of Roland Translated by C. K. [Charle...,,"[Roland, Ganelon, Oliver, King Marsile, Blanca..."
639,She,She by H. Rider Haggard First Published 1886. ...,,"[Laurel Mack, Ellie Mack, Paul Mack, Hanna Mac..."
650,The Wonderful Adventures of Nils,[Transcriber's note: The inconsistent orthogra...,,"[Dunfin, Clement Larsson, Smirre Fox, Morten G..."


In [ ]:
df.loc[df['summary'] == "Summary Section Not found"]

,title,full_text,summary,characters
8,Madame Bovary,Madame Bovary By Gustave Flaubert Translated f...,Summary Section Not found,[]
9,The Divine Comedy,no results found,Summary Section Not found,[]
27,The Aeneid,THE AENEID by Virgil Translated by John Dryden...,Summary Section Not found,[]
30,Tristram Shandy,no results found,Summary Section Not found,[]
31,Leaves of Grass,"LEAVES OF GRASS By Walt Whitman Come, said my ...",Summary Section Not found,[]
...,...,...,...,...
646,Capital of Pain,no results found,Summary Section Not found,[]
650,The Wonderful Adventures of Nils,[Transcriber's note: The inconsistent orthogra...,Summary Section Not found,[]
670,The Continental Op,no results found,Summary Section Not found,[]
679,Kabbalah,no results found,Summary Section Not found,[]


In [ ]:
from google.colab import files

df_good.to_csv('booklist.csv', sep ='~') 
files.download('booklist.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_chars["characters"] = df_chars["characters"].map(lambda x: "_".join(x))

In [ ]:
df_chars = df_chars.drop(columns = "summary")

In [ ]:
from google.colab import files

df_chars.to_csv('charlists.csv', sep ='~') 
files.download('charlists.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>